# RAG Com modelos de linguagem pré-treinados

## Seção 1: Instalação de Bibliotecas
Instalação das bibliotecas necessárias para o projeto. O objetivo é configurar o ambiente com as dependências adequadas para trabalhar com vetores, embeddings e LLMs (Modelos de Linguagem Grande).


In [1]:
!pip install llama-index
!pip install llama-index-vector-stores-chroma
!pip install llama-index-llms-ollama
!pip install llama-index-embeddings-huggingface
!pip install huggingface-hub
!pip install torch
!pip install chromadb
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing insta

## Seção 2: Configuração do LLM e Embeddings
Importação das bibliotecas e configuração inicial do modelo de linguagem (LLM) Ollama e do modelo de embeddings HuggingFace.
Isso permitirá o processamento de linguagem natural e a extração de representações vetoriais dos textos.


In [2]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings, Document, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.storage.storage_context import StorageContext
import chromadb
from pathlib import Path
import pandas as pd

llm = Ollama(model="phi3.5:latest", request_timeout=120.0)
Settings.llm = llm

# Modelo de embeddings
Settings.embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/LaBSE"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

## Seção 3: Configuração do Armazenamento Vetorial
Esta seção configura o armazenamento vetorial utilizando ChromaDB para armazenar e recuperar vetores de documentos.
O armazenamento vetorial é usado aqui para a indexação e recuperação de textos de maneira eficiente.

In [21]:
from chromadb.config import Settings as SettingsChroma
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core import VectorStoreIndex, Settings
from llama_index.readers.file import CSVReader

import chromadb


# Inicializa o cliente ChromaDB e cria uma coleção para os vetores
client = chromadb.Client()
chroma_collection = client.create_collection(name="brastel_faq")

# Cria o armazenamento vetorial e o contexto de armazenamento
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = Settings

UniqueConstraintError: Collection brastel_faq already exists

## Seção 4: Carregamento de Dados e Criação de Documentos
Nesta seção, os dados são carregados de um arquivo CSV contendo perguntas e respostas.
Em seguida, os documentos são criados a partir dos dados carregados.


In [22]:
from llama_index.core import Document
from pathlib import Path
import pandas as pd

# Caminho para o arquivo CSV
csv_file_path = Path("/content/faq.csv")
df = pd.read_csv(csv_file_path)

documents = []
for index, row in df.iterrows():
    pergunta = str(row['Pergunta']).strip()
    resposta = str(row['Resposta']).strip()
    content = f"Pergunta: {pergunta}\nResposta: {resposta}"
    doc = Document(text=content, metadata={"Pergunta": pergunta, "Resposta": resposta, "ID": index})
    documents.append(doc)

print(f"Número total de documentos criados: {len(documents)}")

Número total de documentos criados: 37


## Seção 5: Criação do Índice Vetorial
Após criar os documentos, o índice vetorial é gerado para possibilitar a busca de informações relevantes com base nas perguntas.


In [23]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context)

## Seção 6: Consultas e Recuperação de Resultados
Nesta seção, um motor de consulta é configurado para realizar perguntas ao índice vetorial.
As respostas mais relevantes são recuperadas com base nos embeddings criados.



In [24]:
query_engine = index.as_query_engine()

# Pergunta de exemplo a ser enviada ao índice
question = "posso criar mais de uma conta? tenho pessoas da minha familia que também querem usar"
retriever = index.as_retriever()

results = retriever.retrieve(question)
print(results[:5])

[NodeWithScore(node=TextNode(id_='b9a92063-53e9-48ba-8a45-81a68ed49cdb', embedding=None, metadata={'Pergunta': 'Eu posso ter mais de um nome de usuário ou conta?', 'Resposta': 'Não, só é permitido um nome de usuário por cliente.', 'ID': 28}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7286234c-85bc-4bc9-aca7-d51b70aa7503', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'Pergunta': 'Eu posso ter mais de um nome de usuário ou conta?', 'Resposta': 'Não, só é permitido um nome de usuário por cliente.', 'ID': 28}, hash='77a12e956fb7ce333374151b81794596f439acfe04b659da33b8d08f77f439d8')}, text='Pergunta: Eu posso ter mais de um nome de usuário ou conta?\nResposta: Não, só é permitido um nome de usuário por cliente.', mimetype='text/plain', start_char_idx=0, end_char_idx=121, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.464495677628

## Seção 7: Geração de Contexto e Resposta
Aqui, os metadados dos resultados recuperados são usados para gerar uma resposta clara e concisa, diretamente em português.
Aresposta é gerada com base nas referências relevantes do arquivo de FAQ.



In [27]:
import requests
import pandas as pd
from pathlib import Path

# Path to the CSV file
csv_file_path = Path("/content/faq.csv")

# Define the alternative API URL
url = 'https://api.exchangerate-api.com/v4/latest/JPY'  # Using a different API endpoint

# Fetch the exchange rate for IENE
try:
    response = requests.get(url)
    if response.status_code == 200:
        exchange_data = response.json()

        # Get the exchange rates for USD and BRL
        iene_to_usd = exchange_data.get('rates', {}).get('USD')
        brl_to_usd = exchange_data.get('rates', {}).get('BRL')

        if iene_to_usd and brl_to_usd:
            # Calculate IENE to BRL
            iene_to_brl = iene_to_usd / brl_to_usd
            print(f"IENE to BRL exchange rate: {iene_to_brl}")

            # Insert this data into the CSV file
            try:
                # Read the existing CSV file
                if csv_file_path.exists():
                    df = pd.read_csv(csv_file_path)
                else:
                    # If the CSV doesn't exist, create an empty DataFrame with the correct columns
                    df = pd.DataFrame(columns=['currency', 'rate', 'date'])

                # Add the new data
                new_data = pd.DataFrame({'currency': ['IENE'], 'rate': [iene_to_brl], 'date': [pd.Timestamp.now()]})

                # Concatenate the new data with the existing DataFrame
                df = pd.concat([df, new_data], ignore_index=True)

                # Save back to the CSV file
                df.to_csv(csv_file_path, index=False)
                print("Exchange rate successfully inserted into the CSV file.")
            except Exception as e:
                print(f"An error occurred while writing to the CSV file: {e}")
        else:
            print("Exchange rate for IENE or BRL not found in the API response.")
    else:
        print(f"Failed to fetch exchange rates. Status code: {response.status_code}")
except requests.ConnectionError:
    print("Failed to connect to the exchange rate API.")


IENE to BRL exchange rate: 0.1825857519788918
Exchange rate successfully inserted into the CSV file.


In [32]:
context_list = [str(n.metadata) for n in results]
print(context_list)

prompt = (
    "Using the references below, provide a direct and natural response to the user's question in Portuguese.\n"
    "Ensure that the answer is grammatically correct and flows naturally in Portuguese.\n"
    "To enhance the response, cite the 'Resposta' number that addresses the user's question.\n"
    "Answer the user's question DIRECTLY in a concise and respectful manner.\n"
    "Only reference answers that are relevant to the user's question.\n"
    "Data:\n\n"
    + "\n\n".join(context_list + [question])
)

# Gera a resposta usando a LLM configurada
response = llm.complete(prompt)
print(str(response))

["{'Pergunta': 'Eu posso ter mais de um nome de usuário ou conta?', 'Resposta': 'Não, só é permitido um nome de usuário por cliente.', 'ID': 28}"]


ConnectError: [Errno 111] Connection refused